<a href="https://colab.research.google.com/github/danielelbrecht/mirna/blob/master/refocus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import sklearn
from keras.datasets import cifar10
import time

# Keras
from keras.models import Model
from keras.layers import Input, Dropout, Dense, Permute, Flatten, Conv2D, MaxPooling2D
from keras import regularizers, optimizers, utils

In [0]:
#Load data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

170500096/170498071 [==============================] - 49s 0us/step


In [0]:
from keras.utils import to_categorical

# Preprocess data
train_images = train_images.reshape((50000, 32, 32, 3)) / 255.0
test_images = test_images.reshape((10000, 32, 32, 3)) / 255.0

train_labels_categorical = utils.to_categorical(train_labels, num_classes=10, dtype='float32')
test_labels_categorical = utils.to_categorical(test_labels, num_classes=10, dtype='float32')

In [0]:
# Build model

# Input layer
inputs = Input(shape=(32, 32, 3), name='inputs')

# First convolutional module
conv_1 = Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(32,32,3))(inputs)
pool_1 = MaxPooling2D(pool_size=(2,2))(conv_1)
do_1 = Dropout(0.2)(pool_1)

# Second convolutional module
conv_2 = Conv2D(filters=256, kernel_size=(3,3), activation='relu')(do_1)
pool_2 = MaxPooling2D(pool_size=(2,2))(conv_2)
do_2 = Dropout(0.2)(pool_2)

# Flatten layer
flat = Flatten()(do_2)

# Dense layers
dense_1 = Dense(256, activation='relu')(flat)
do_3 = Dropout(0.2)(dense_1)
dense_2 = Dense(128, activation='relu')(do_3)
dense_3 = Dense(64, activation='relu')(dense_2)

# Prediction layer
softmax = Dense(10, activation='softmax')(dense_3)

# Compile model
model = Model(inputs=inputs, outputs=softmax)

model.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

model.summary()

config = model.get_config()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 256)         0         
___________________________

In [0]:
# Define hyper parameters
epochs = 25

In [0]:
# Reset model
model = Model.from_config(config)
model.compile(optimizer=opt,
               loss='categorical_crossentropy',
               metrics=['accuracy'])

initial_time = time.time()

history = model.fit(train_images, 
                    train_labels_categorical, 
                    epochs=40,
                    batch_size = 128,
                    validation_data=(test_images, test_labels_categorical))

print('Trained for 40 epochs.  Time elapsed: {} seconds'.format(time.time()-initial_time))

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 10s 206us/step - loss: 1.7312 - acc: 0.3750 - val_loss: 1.2713 - val_acc: 0.5526
Epoch 2/40
50000/50000 [==============================] - 10s 192us/step - loss: 1.2506 - acc: 0.5616 - val_loss: 1.0752 - val_acc: 0.6285
Epoch 3/40
50000/50000 [==============================] - 10s 203us/step - loss: 1.0476 - acc: 0.6373 - val_loss: 0.9643 - val_acc: 0.6672
Epoch 4/40
50000/50000 [==============================] - 10s 196us/step - loss: 0.9122 - acc: 0.6845 - val_loss: 0.9423 - val_acc: 0.6828
Epoch 5/40
50000/50000 [==============================] - 10s 196us/step - loss: 0.8133 - acc: 0.7195 - val_loss: 0.9798 - val_acc: 0.6655
Epoch 6/40
50000/50000 [==============================] - 10s 193us/step - loss: 0.7262 - acc: 0.7489 - val_loss: 0.8303 - val_acc: 0.7211
Epoch 7/40
50000/50000 [==============================] - 10s 193us/step - loss: 0.6503 - acc: 0.7731 - val_loss: 0.

In [0]:
def refocus(model, x, y):
  predictions = model.predict(x)
  mask = np.ones(len(predictions), dtype=bool)
  
  for i in range(len(predictions)):
    if np.argmax(predictions[i]) == np.argmax(y[i]) and max(predictions[i]) > 0.99:
      mask[i] = 0
      
  new_x = x[mask]
  new_y = y[mask]
  f = float(len(new_x)/len(mask))
  
  return new_x, new_y, f


In [0]:
# model with checkpoints
from keras import optimizers

lr = 0.001
opt = optimizers.RMSprop(lr=lr, rho=0.9, epsilon=None, decay=0.0)


# Reset model
model = Model.from_config(config)
model.compile(optimizer=opt,
               loss='categorical_crossentropy',
               metrics=['accuracy'])


initial_time = time.time()

x = train_images
y = train_labels_categorical

for i in range(8):
  history = model.fit(x, 
                      y, 
                      epochs=5,
                      batch_size = 128,
                      validation_data=(test_images, test_labels_categorical))
  x, y, f = refocus(model, x, y)
  
  # recompile with new learning rate
  lr = lr*f
  print('New learning rate set to {}'.format(lr))
  model.compile(optimizer=optimizers.RMSprop(lr=lr, rho=0.9, epsilon=None, decay=0.0),
               loss='categorical_crossentropy',
               metrics=['accuracy'])
  
  
  

print('Trained for 40 epochs.  Time elapsed: {} seconds'.format(time.time()-initial_time))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 15s 294us/step - loss: 1.7206 - acc: 0.3779 - val_loss: 1.5447 - val_acc: 0.4410
Epoch 2/5
50000/50000 [==============================] - 9s 174us/step - loss: 1.2358 - acc: 0.5633 - val_loss: 1.0917 - val_acc: 0.6225
Epoch 3/5
50000/50000 [==============================] - 9s 176us/step - loss: 1.0399 - acc: 0.6373 - val_loss: 1.1655 - val_acc: 0.5962
Epoch 4/5
50000/50000 [==============================] - 9s 176us/step - loss: 0.9032 - acc: 0.6873 - val_loss: 0.9269 - val_acc: 0.6774
Epoch 5/5
50000/50000 [==============================] - 9s 177us/step - loss: 0.8028 - acc: 0.7221 - val_loss: 0.9018 - val_acc: 0.6861
New learning rate set to 0.00090116
Train on 45058 samples, validate on 10000 samples
Epoch 1/5
45058/45058 [==============================] - 9s 192us/step - loss: 0.7657 - acc: 0.7358 - val_loss: 0.8564 - val_acc: 